In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, desc

spark = SparkSession.builder.appName("ShopAnalysis").getOrCreate()

# ข้อมูลจำลอง
data = [
    ("TX1", "Somchai", "Electronics", 25000, "Completed"),
    ("TX2", "Somsri",  "Clothing",    500,   "Completed"),
    ("TX3", "Somchai", "Clothing",    1200,  "Cancelled"), # <-- รายการนี้ต้องโดนตัดออก
    ("TX4", "John",    "Electronics", 15000, "Completed"),
    ("TX5", "Somsri",  "Beauty",      2000,  "Completed"),
    ("TX6", "John",    "Beauty",      800,   "Cancelled"), # <-- รายการนี้ต้องโดนตัดออก
    ("TX7", "Somsri",  "Electronics", 5000,  "Completed")
]
columns = ["OrderID", "Customer", "Category", "Price", "Status"]

# แปลงเป็น PySpark DataFrame
df = spark.createDataFrame(data, columns)

print("--- ข้อมูลดิบทั้งหมด ---")
df.show()

--- ข้อมูลดิบทั้งหมด ---
+-------+--------+-----------+-----+---------+
|OrderID|Customer|   Category|Price|   Status|
+-------+--------+-----------+-----+---------+
|    TX1| Somchai|Electronics|25000|Completed|
|    TX2|  Somsri|   Clothing|  500|Completed|
|    TX3| Somchai|   Clothing| 1200|Cancelled|
|    TX4|    John|Electronics|15000|Completed|
|    TX5|  Somsri|     Beauty| 2000|Completed|
|    TX6|    John|     Beauty|  800|Cancelled|
|    TX7|  Somsri|Electronics| 5000|Completed|
+-------+--------+-----------+-----+---------+



เจ้านายสั่งงานมา 3 ข้อ:

- Clean Data: ตัดรายการที่ "ยกเลิก" (Cancelled) ออกไป ให้เหลือแค่ยอดที่ขายได้จริง

- Category Report: อยากรู้ว่าสินค้า "หมวดหมู่ไหน" ขายดีที่สุด (มียอดเงินรวมสูงสุด)?

- Find VIP: ใครคือลูกค้าที่มียอดซื้อรวม (ที่สำเร็จแล้ว) สูงที่สุด?

In [2]:
df_clean = df.filter(df.Status == "Completed")
df_clean.show()

+-------+--------+-----------+-----+---------+
|OrderID|Customer|   Category|Price|   Status|
+-------+--------+-----------+-----+---------+
|    TX1| Somchai|Electronics|25000|Completed|
|    TX2|  Somsri|   Clothing|  500|Completed|
|    TX4|    John|Electronics|15000|Completed|
|    TX5|  Somsri|     Beauty| 2000|Completed|
|    TX7|  Somsri|Electronics| 5000|Completed|
+-------+--------+-----------+-----+---------+



In [3]:
df2 = df_clean.groupBy("Category").agg(sum("Price")).alias("Total_Price")
df2.show()

+-----------+----------+
|   Category|sum(Price)|
+-----------+----------+
|Electronics|     45000|
|   Clothing|       500|
|     Beauty|      2000|
+-----------+----------+



In [4]:
df3 = df_clean.groupBy("Customer").agg(sum("Price").alias("Total_Spending")).orderBy(desc("Total_Spending"))
df3.show(1)

+--------+--------------+
|Customer|Total_Spending|
+--------+--------------+
| Somchai|         25000|
+--------+--------------+
only showing top 1 row


In [6]:
df3.write.csv("clean_data", header=True, mode="overwrite")